In [3]:
import os
import numpy as np
import sklearn.metrics as metrics
os.system("")
def sigmoid(z):
    return 1/(1 + np.exp(-z))

BASELINE_PREDICTION_PATH = '/home/vit/Projects/cryptic-nn/src/models/predict/predictions/baseline'
FINETUNED_PREDICTION_PATH = '/home/vit/Projects/cryptic-nn/src/models/predict/predictions/finetuned-multitask-with-ligysis'
GROUND_TRUTH_PATH = '/home/vit/Projects/cryptic-nn/src/models/predict/ground-truth/baseline'
for i in os.listdir(BASELINE_PREDICTION_PATH):
    baseline_prediction = (sigmoid(np.load(os.path.join(BASELINE_PREDICTION_PATH, i))) > 0.95) * 1.0
    finetuned_prediction = (np.load(os.path.join(FINETUNED_PREDICTION_PATH, i)) > 0.7) * 1.0
    ground_truth = np.load(os.path.join(GROUND_TRUTH_PATH, i))
    mcc_baseline = metrics.matthews_corrcoef(ground_truth, baseline_prediction)
    mcc_finetuned = metrics.matthews_corrcoef(ground_truth, finetuned_prediction)
    if mcc_finetuned - mcc_baseline > 0.25 and mcc_finetuned - mcc_baseline < 3:
        print(('\033[31m' if mcc_baseline - mcc_finetuned > 0 else '\033[34m') + f"{i}: MCC Baseline: {mcc_baseline:.4f}, MCC Finetuned: {mcc_finetuned:.4f}")



5uxaA.npy: MCC Baseline: 0.4321, MCC Finetuned: 0.8154
7xgfE.npy: MCC Baseline: 0.1992, MCC Finetuned: 0.4608
5e0vA.npy: MCC Baseline: 0.2410, MCC Finetuned: 0.6908
7x0iB.npy: MCC Baseline: 0.4575, MCC Finetuned: 0.7406
5ysbB.npy: MCC Baseline: 0.4462, MCC Finetuned: 0.7245
2vl2C.npy: MCC Baseline: 0.5599, MCC Finetuned: 0.8335
1i7nA.npy: MCC Baseline: 0.5679, MCC Finetuned: 0.8985
1ksgB.npy: MCC Baseline: 0.0330, MCC Finetuned: 0.5006
3flgA.npy: MCC Baseline: 0.5238, MCC Finetuned: 0.8750
4gpiC.npy: MCC Baseline: 0.1643, MCC Finetuned: 0.6635
4dncB.npy: MCC Baseline: 0.3557, MCC Finetuned: 0.6304
2akaA.npy: MCC Baseline: 0.3543, MCC Finetuned: 0.7042
7kayA.npy: MCC Baseline: 0.2338, MCC Finetuned: 0.5263
7yjcA.npy: MCC Baseline: -0.0298, MCC Finetuned: 0.2756
1h13A.npy: MCC Baseline: 0.1188, MCC Finetuned: 0.5079
5ey7B.npy: MCC Baseline: 0.1773, MCC Finetuned: 0.4697
1xgdA.npy: MCC Baseline: 0.5405, MCC Finetuned: 0.7925
2vyrA.npy: MCC Baseline: 0.3214, MCC Finetuned: 0.7085
6du4A.npy

In [1]:
import sys
import pymol
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr
from pymol import cmd

CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'


Gtk-Message: 16:25:40.611: Failed to load module "pk-gtk-module"
Cannot open file '/home/vit/Projects/cryptic-nn/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory
Cannot open file '/home/vit/Projects/cryptic-nn/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory


In [7]:
from biotite.structure import get_residues, get_residue_starts, spread_residue_wise
from biotite.structure.io.pdbx import get_structure
import biotite.structure.io.pdbx as pdbx
import biotite.database.rcsb as rcsb
import biotite

proteins = [#'5e0vA', maybee good
             #'1ksgB', 
             #'4gpiC', maybeee
            # '5uxaA', 
            #'5e0vA',
           #  '1i7nA',
           # '1ksgB',
            #'3flgA',
           # '4gpiC',
           #  '2akaA',
            # '7yjcA', # missed by baseline!
            # '1h13A',
            # '2vyrA',
            # '6du4A',
           #  '6w10A',
            # '5uxaA',
# '7xgfE',
# '5e0vA',
# '7x0iB',
# '5ysbB',
# '2vl2C',
# '1i7nA',
'1ksgB', # also missed by baseline!
'3flgA',
'4gpiC',
'4dncB',
'2akaA',
'7kayA',
'7yjcA',
'1h13A',
'5ey7B',
'1xgdA',
'2vyrA',
'6du4A',
'6w10A',
] 

# visualize = 'baseline'
visualize = 'finetuning'

for protein in proteins:
    apo_structure = protein[:4]
    chain_id = protein[4:]
    prediction_filename = f'{protein}.npy'

    cif_file_path = rcsb.fetch(apo_structure, "cif", target_path=CIF_FILES)
    cif_file = pdbx.CIFFile.read(cif_file_path)
    auth = get_structure(cif_file, model=1)

    auth = auth[
            (auth.chain_id == chain_id) &
            (biotite.structure.filter_amino_acids(auth))]
    auth_residues_only = auth[get_residue_starts(auth)]

    baseline_prediction = (sigmoid(np.load(os.path.join(BASELINE_PREDICTION_PATH, prediction_filename))) > 0.95) * 1
    finetuned_prediction = (np.load(os.path.join(FINETUNED_PREDICTION_PATH, prediction_filename)) > 0.7) * 1.0
    ground_truth = np.load(os.path.join(GROUND_TRUTH_PATH, prediction_filename))

    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein)
    cmd.zoom(protein)
    cmd.color('brown', protein)

    if visualize == 'baseline':
        positive_indices = np.where(baseline_prediction == 1)
    else:
        positive_indices = np.where(finetuned_prediction == 1)

    binding_residues = np.where(ground_truth == 1)
    intersection = np.intersect1d(binding_residues, positive_indices)

    if len(positive_indices) > 0:
        cmd.color('red', f'{protein} and resid {"+".join([str(i) for i in auth_residues_only.res_id[positive_indices]])}')
    if len(binding_residues) > 0:
        cmd.color('yellow', f'{protein} and resid {"+".join([str(i) for i in auth_residues_only.res_id[binding_residues]])}')
    if len(intersection) > 0:
        cmd.color('green', f'{protein} and resid {"+".join([str(i) for i in auth_residues_only.res_id[intersection]])}')
    cmd.show('surface')

    decision = input(">Press Enter for the next protein...\n")   
    if decision == 'q':
        break


 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
PyMOL>set opaque_background, 0
 Setting: opaque_background set to off.
PyMOL>png /home/vit/Downloads/finetuned-RAS.png, 0, 0, -1, ray=1
 Ray: render time: 0.65 sec. = 5501.1 frames/hour (1.27 sec. accum.).
 ScenePNG: wrote 640x508 pixel image to file "/home/vit/Downloads/finetuned-RAS.png".
